In [4]:
from job_supervisor_client import *

communitySummaJob = Job('summa', url="localhost", port=3000, isJupyter=True)

In [5]:
communitySummaJob.submit()

✅ job registered with ID: 1597718558amMh


In [6]:
communitySummaJob.events(liveOutput=True)

📮Job ID: 1597718558amMh
📍Destination: summa



types,message,time
JOB_QUEUED,"job [1597718558amMh] is queued, waiting for registration",2020-08-18T02:42:38.332Z
JOB_REGISTERED,"job [1597718558amMh] is registered with the supervisor, waiting for initialization",2020-08-18T02:42:38.365Z
SUMMA_RESOURCES_DOWNLOADED,downloaded resources from Hydroshare for SUMMA,2020-08-18T02:42:48.395Z
SUMMA_HPC_CONNECTED,connected to HPC,2020-08-18T02:42:48.395Z
SUMMA_HPC_SUBMITTED,submitted SUMMA job to HPC,2020-08-18T02:42:48.395Z
JOB_INITIALIZED,initialized SUMMA job in HPC job queue with remote_id 3593739,2020-08-18T02:42:48.395Z
JOB_ENDED,SUMMA job with remote_id 3593739 completed,2020-08-18T02:43:03.922Z


In [7]:
import os

communitySummaJob.download(os.getcwd())